## Lectura del fichero de estado de vacunación publicado por Sanidad en  
* https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm


In [1]:
# importamos pandas, numpy y datetime
# previamente ha sido necesatio instalar el motor odfpy con un pip3 install odfpy
import pandas as pd
import numpy as np
from datetime import datetime,timedelta

Version branch cambios 18 enero

preparamos nombre del fichero csv, dentro del directorio csv. Preparamos dos ficheros:
  * `estado_vacunacion_.csv` , última versión del mismo con el acumulado de todos los informes publicados
  * `estado_vacunacion_yearmonthday.csv`, con la versión del día en cuestión, solo con la información de ese día

In [2]:
path_directorio='/Users/mharias/documents/proyectos/covid/vacunacion/csv/'
nombre_root_fichero = 'estado_vacunacion_{}'
fecha_informe = datetime.today()
#en el caso de queramos bajar otra fecha diferente a 'hoy'
#fecha_informe = datetime(year=2021,month=2,day=19)
fecha_informe_str=fecha_informe.strftime('%Y%m%d') #fecha formateada para nombre fichero
fecha_informe_pandas = fecha_informe.strftime('%-d/%-m/%y') #idem para el campo pandas
nombre_fichero_con_fecha = path_directorio + nombre_root_fichero.format(fecha_informe_str)
nombre_fichero_acumulado = path_directorio + nombre_root_fichero.format('')

Enlace al fichero en red en formato ods

In [3]:
enlace_bruto = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_{}.ods'
enlace = enlace_bruto.format(fecha_informe_str)

Leemos el fichero

In [4]:
try:
    datos_row = pd.read_excel(enlace)
    print ('Ok, descargado {}'.format(enlace))
except:
    print ('No esta listo aun')

Ok, descargado https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/Informe_Comunicacion_20210301.ods


Le echamos un vistazo para asegurar que ha ido todo bien

In [5]:
datos_row

,Unnamed: 0,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,662310,36300,144200,842810,769899,0.913491,228739,2021-02-28
1,Aragón,112635,6300,22600,141535,130122,0.919363,46822,2021-02-27
2,Asturias,112055,6300,17400,135755,117129,0.862797,39215,2021-02-28
3,Baleares,56520,3200,19900,79620,61342,0.770435,21950,2021-02-28
4,Canarias,129840,7000,37100,173940,146120,0.840060,51446,2021-02-28
5,Cantabria,55725,2900,9900,68525,59144,0.863101,20433,2021-02-27
6,Castilla y Leon,246585,13500,40800,300885,257479,0.855739,89946,2021-02-28
7,Castilla La Mancha,167175,9200,34800,211175,162939,0.771583,68732,2021-02-28
8,Cataluña,570410,31400,132700,734510,650327,0.885389,188109,2021-02-28
9,C. Valenciana,288010,15700,86200,389910,323505,0.829691,102016,2021-02-28


..hacemos una copia para hacerle las transformaciones necesarias:

In [6]:
datos_hoy = datos_row.copy()

Primero renombramos columnas, añadimos luego una columna con la fecha del informe `date_pub`, y finalmente añadimos fuente y origen de datos para asegurar trazabilidad de los datos:

In [7]:
datos_hoy.columns=['ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas','Dosis administradas','% sobre entregadas','Total pauta completada','Fecha de la ultima vacuna registrada']

In [8]:
datos_hoy['date_pub']=fecha_informe_pandas
datos_hoy['source_name']='Ministerio de Sanidad'
datos_hoy['source'] = enlace


vamos a convertir el formato de `Fecha de la ultima vacuna registrada` para que aparezca como día/mes/año sin añadir ceros de pad..de ahi el signo menos en el caso de dia y mes (%-d/%-m). La excepción se añade para la línea de Totales, que no lleva fecha asociada:

In [9]:
def convierte(x):
    campo =  x['Fecha de la ultima vacuna registrada']
    try:
        fecha = campo.strftime('%-d/%-m/%y')
        return fecha
    except:
        return campo

In [10]:
datos_hoy['Fecha de la ultima vacuna registrada'] = datos_hoy.apply(convierte,axis=1)

..y ordenamos..

In [11]:
datos_hoy

,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,date_pub,source_name,source
0,Andalucía,662310,36300,144200,842810,769899,0.913491,228739,28/2/21,1/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,Aragón,112635,6300,22600,141535,130122,0.919363,46822,27/2/21,1/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,Asturias,112055,6300,17400,135755,117129,0.862797,39215,28/2/21,1/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,Baleares,56520,3200,19900,79620,61342,0.770435,21950,28/2/21,1/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,Canarias,129840,7000,37100,173940,146120,0.840060,51446,28/2/21,1/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,Cantabria,55725,2900,9900,68525,59144,0.863101,20433,27/2/21,1/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,Castilla y Leon,246585,13500,40800,300885,257479,0.855739,89946,28/2/21,1/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,Castilla La Mancha,167175,9200,34800,211175,162939,0.771583,68732,28/2/21,1/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,Cataluña,570410,31400,132700,734510,650327,0.885389,188109,28/2/21,1/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,C. Valenciana,288010,15700,86200,389910,323505,0.829691,102016,28/2/21,1/3/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [12]:
datos_hoy=datos_hoy[['date_pub','ccaa','Dosis entregadas Pfizer',
                    'Dosis entregadas Moderna','Dosis entregadas AstraZeneca','Dosis entregadas',
                    'Dosis administradas','% sobre entregadas',
                    'Total pauta completada',
                    'Fecha de la ultima vacuna registrada','source_name','source']]

Hemos visto algún informe en el que se añaden espacios al nombre de Comunidad, nos aseguramos aquí de eliminarlos en caso de que vuelvan a aparecer:

In [13]:
datos_hoy['ccaa'] = datos_hoy['ccaa'].apply(lambda x : x.rstrip().lstrip())

In [14]:
datos_hoy['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

In [15]:
datos_hoy

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Fecha de la ultima vacuna registrada,source_name,source
0,1/3/21,Andalucía,662310,36300,144200,842810,769899,0.913491,228739,28/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
1,1/3/21,Aragón,112635,6300,22600,141535,130122,0.919363,46822,27/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
2,1/3/21,Asturias,112055,6300,17400,135755,117129,0.862797,39215,28/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
3,1/3/21,Baleares,56520,3200,19900,79620,61342,0.770435,21950,28/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
4,1/3/21,Canarias,129840,7000,37100,173940,146120,0.840060,51446,28/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
5,1/3/21,Cantabria,55725,2900,9900,68525,59144,0.863101,20433,27/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
6,1/3/21,Castilla y Leon,246585,13500,40800,300885,257479,0.855739,89946,28/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
7,1/3/21,Castilla La Mancha,167175,9200,34800,211175,162939,0.771583,68732,28/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
8,1/3/21,Cataluña,570410,31400,132700,734510,650327,0.885389,188109,28/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
9,1/3/21,C. Valenciana,288010,15700,86200,389910,323505,0.829691,102016,28/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


guardamos el fichero de hoy en el directorio csv

In [16]:
datos_hoy.to_csv(nombre_fichero_con_fecha+'.csv',index = False)

y ahora bajamos el acumulado actual para añadirle los datos de hoy, y volverlo a subir 

In [17]:
datos_acumulados = pd.read_csv(nombre_fichero_acumulado+'.csv')

In [18]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
775,26/2/21,Navarra,55725.0,2900.0,11200.0,69825,57034,0.816813,21274.0,NaN,25/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
776,26/2/21,País Vasco,148410.0,7900.0,37900.0,194210,122558,0.631059,46157.0,NaN,25/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
777,26/2/21,Ceuta,4320.0,400.0,1500.0,6220,6272,1.008360,1136.0,NaN,25/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
778,26/2/21,Melilla,3150.0,400.0,1500.0,5050,4710,0.932673,1267.0,NaN,25/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


concatenamos...

In [19]:
datos_acumulados=pd.concat([datos_acumulados,datos_hoy],ignore_index=True)

In [20]:
datos_acumulados.columns

Index(['date_pub', 'ccaa', 'Dosis entregadas Pfizer',
       'Dosis entregadas Moderna', 'Dosis entregadas AstraZeneca',
       'Dosis entregadas', 'Dosis administradas', '% sobre entregadas',
       'Total pauta completada', 'Última fecha de actualización de datos',
       'Fecha de la ultima vacuna registrada', 'source_name', 'source'],
      dtype='object')

In [21]:
datos_acumulados = datos_acumulados[['date_pub', 'ccaa','Dosis entregadas Pfizer','Dosis entregadas Moderna',
                                     'Dosis entregadas AstraZeneca',
                                     'Dosis entregadas', 'Dosis administradas','% sobre entregadas',
                                     'Total pauta completada','Última fecha de actualización de datos',
                                     'Fecha de la ultima vacuna registrada','source_name', 'source']]

In [22]:
datos_acumulados

,date_pub,ccaa,Dosis entregadas Pfizer,Dosis entregadas Moderna,Dosis entregadas AstraZeneca,Dosis entregadas,Dosis administradas,% sobre entregadas,Total pauta completada,Última fecha de actualización de datos,Fecha de la ultima vacuna registrada,source_name,source
0,4/1/21,Andalucía,NaN,NaN,NaN,140295,25809,0.183962,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 19,https://www.mscbs.gob.es/profesionales/saludPu...
1,4/1/21,Aragón,NaN,NaN,NaN,23715,2004,0.084503,NaN,2/1/21,2/1/21,Ministerio de Sanidad - Vacuna Covid 20,https://www.mscbs.gob.es/profesionales/saludPu...
2,4/1/21,Asturias,NaN,NaN,NaN,23720,9380,0.395447,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 21,https://www.mscbs.gob.es/profesionales/saludPu...
3,4/1/21,Baleares,NaN,NaN,NaN,8940,153,0.017114,NaN,28/12/20,28/12/20,Ministerio de Sanidad - Vacuna Covid 22,https://www.mscbs.gob.es/profesionales/saludPu...
4,4/1/21,Canarias,NaN,NaN,NaN,20835,4846,0.232589,NaN,3/1/21,3/1/21,Ministerio de Sanidad - Vacuna Covid 23,https://www.mscbs.gob.es/profesionales/saludPu...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,1/3/21,Navarra,55725.0,2900.0,11200.0,69825,60697,0.869273,21719.0,NaN,28/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
796,1/3/21,País Vasco,148410.0,7900.0,37900.0,194210,127629,0.657170,47492.0,NaN,28/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
797,1/3/21,Ceuta,4320.0,400.0,1500.0,6220,6273,1.008521,1136.0,NaN,25/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...
798,1/3/21,Melilla,3150.0,400.0,1500.0,5050,4710,0.932673,1267.0,NaN,25/2/21,Ministerio de Sanidad,https://www.mscbs.gob.es/profesionales/saludPu...


In [23]:
datos_acumulados['ccaa'].unique()

array(['Andalucía', 'Aragón', 'Asturias', 'Baleares', 'Canarias',
       'Cantabria', 'Castilla y Leon', 'Castilla La Mancha', 'Cataluña',
       'C. Valenciana', 'Extremadura', 'Galicia', 'La Rioja', 'Madrid',
       'Murcia', 'Navarra', 'País Vasco', 'Ceuta', 'Melilla', 'Totales'],
      dtype=object)

y volvemos a subirlo..

In [24]:
datos_acumulados.to_csv(nombre_fichero_acumulado+'.csv',index=False)